In [2]:
import pandas as pd
import time
import ast

In [36]:
# Load the hotel reviews from CSV
start = time.time()
df = pd.read_csv('../data/Hotel_Reviews.csv')
end = time.time()
print("loading took " + str(round(end - start, 2)) + " seconds")


loading took 3.22 seconds


In [37]:
# dropping columns we will not use:
df.drop(["lat", "lng"], axis = 1, inplace=True)


In [12]:
def replace_address(row):
    if "Netherlands" in row["Hotel_Address"]:
        return "Amsterdam, Netherlands"
    elif "Barcelona" in row["Hotel_Address"]:
        return "Barcelona, Spain"
    elif "United Kingdom" in row["Hotel_Address"]:
        return "London, United Kingdom"
    elif "Milan" in row["Hotel_Address"]:        
        return "Milan, Italy"
    elif "France" in row["Hotel_Address"]:
        return "Paris, France"
    elif "Vienna" in row["Hotel_Address"]:
        return "Vienna, Austria" 
    else:
        return row.Hotel_Address

In [38]:
# Replace all the addresses with a shortened, more useful form
df["Hotel_Address"] = df.apply(replace_address, axis = 1)

In [39]:
# Drop `Additional_Number_of_Scoring`
df.drop(["Additional_Number_of_Scoring"], axis = 1, inplace=True)
# Replace `Total_Number_of_Reviews` and `Average_Score` with our own calculated values
# df.Total_Number_of_Reviews = df.groupby('Hotel_Name').transform('count')   # 原代码中这一句有问题，我替换成下面这一句了
df.groupby('Hotel_Name').size().reset_index(name='Total_Number_of_Reviews')
df.Average_Score = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)


In [43]:
df.Tags

0         [' Leisure trip ', ' Couple ', ' Duplex Double...
1         [' Leisure trip ', ' Couple ', ' Duplex Double...
2         [' Leisure trip ', ' Family with young childre...
3         [' Leisure trip ', ' Solo traveler ', ' Duplex...
4         [' Leisure trip ', ' Couple ', ' Suite ', ' St...
                                ...                        
515733    [' Leisure trip ', ' Family with older childre...
515734    [' Leisure trip ', ' Family with young childre...
515735    [' Leisure trip ', ' Family with older childre...
515736    [' Leisure trip ', ' Group ', ' Standard Tripl...
515737    [' Leisure trip ', ' Family with young childre...
Name: Tags, Length: 515738, dtype: object

In [7]:
# Process the Tags into new columns
# The file Hotel_Reviews_Tags.py, identifies the most important tags
# Leisure trip, Couple, Solo traveler, Business trip, Group combined with Travelers with friends, 
# Family with young children, Family with older children, With a pet
# 休闲旅行
df["Leisure_trip"] = df.Tags.apply(lambda tag: 1 if "Leisure trip" in tag else 0)
# 夫妻
df["Couple"] = df.Tags.apply(lambda tag: 1 if "Couple" in tag else 0)
# 独自
df["Solo_traveler"] = df.Tags.apply(lambda tag: 1 if "Solo traveler" in tag else 0)
# 商务
df["Business_trip"] = df.Tags.apply(lambda tag: 1 if "Business trip" in tag else 0)
# 团体
df["Group"] = df.Tags.apply(lambda tag: 1 if "Group" in tag or "Travelers with friends" in tag else 0)
# 有婴儿
df["Family_with_young_children"] = df.Tags.apply(lambda tag: 1 if "Family with young children" in tag else 0)
# 有大孩子的家庭
df["Family_with_older_children"] = df.Tags.apply(lambda tag: 1 if "Family with older children" in tag else 0)
# 带宠物
df["With_a_pet"] = df.Tags.apply(lambda tag: 1 if "With a pet" in tag else 0)


In [44]:
# No longer need any of these columns
df.drop(["Review_Date", "Review_Total_Negative_Word_Counts", "Review_Total_Positive_Word_Counts", "days_since_review", "Total_Number_of_Reviews_Reviewer_Has_Given"], axis = 1, inplace=True)


In [46]:
# Saving new data file with calculated columns
start = time.time()
print("Saving results to Hotel_Reviews_Filtered.csv")
df.to_csv(r'../data/Hotel_Reviews_Filtered.csv', index = False)
end = time.time()
print("Filtering took " + str(round(end - start, 2)) + " seconds")


Saving results to Hotel_Reviews_Filtered.csv
Filtering took 6.37 seconds
